In [ ]:
# 13장 네트워크와 프로세스 간 통신 다루기
# -------------------------------------
# 하나의 컴퓨터에서 다른 컴퓨터와 데이터를 주고받으려면 통신이 필요하다. 
# 이번 장에서는 비동기, 소켓, 시그널과 관련된 통신 프로그램을 작성할 때 사용하는 파이썬 모듈을 알아본다.

## 071 비동기 방식으로 프로그래밍하려면? ― asyncio


In [ ]:
# 071 비동기 방식으로 프로그래밍하려면? ― asyncio

# asyncio는 async/await 구문을 사용하여 동시성 코드를 작성할 수 있게 해주는 모듈로, asyncio를 사용하면 단일 스레드 작업을 병렬로 처리할 수 있다.

# 파이썬 3.7 버전 이상부터 사용할 수 있다.

# 문제
# 다음은 서로 다른 입력 값으로 sum() 함수를 2번 수행하여 결괏값을 출력하는 파이썬 프로그램이다.

# import time

# def sleep():
#     time.sleep(1)

# def sum(name, numbers):
#     start = time.time()
#     total = 0
#     for number in numbers:
#         sleep()
#         total += number
#         print(f'작업중={name}, number={number}, total={total}')
#     end = time.time()
#     print(f'작업명={name}, 걸린시간={end-start}')
#     return total


# def main():
#     start = time.time()

#     result1 = sum("A", [1, 2])
#     result2 = sum("B", [1, 2, 3])

#     end = time.time()
#     print(f'총합={result1+result2}, 총시간={end-start}')


# if __name__ == "__main__":
#     main()

# sum() 함수에서 입력 값을 하나씩 더할 때마다 sleep() 함수에 의해 1초씩 시간이 소요된다. 이 프로그램을 실행한 결과는 다음과 같다.

# 작업중=A, number=1, total=1
# 작업중=A, number=2, total=3
# 작업명=A, 걸린시간=2.000162124633789
# 작업중=B, number=1, total=1
# 작업중=B, number=2, total=3
# 작업중=B, number=3, total=6
# 작업명=B, 걸린시간=3.0002427101135254
# 총합=9, 총시간=5.0004048347473145

# A 작업 2초, B 작업 3초 등 모두 5초의 시간이 걸리고 sum() 함수를 2번 호출한 결과의 총합은 9임을 알 수 있다.

# sum() 함수를 비동기 방식으로 호출하면 실행 시간을 줄일 수 있을 것이다. 
# 이 프로그램을 파이썬 비동기 모듈인 asyncio를 사용하여 비동기 방식으로 바꾸려면 어떻게 해야 할까?

In [1]:
import time

def sleep():
    time.sleep(1)
    
def sum(name, numbers):
    start = time.time()
    total = 0
    for number in numbers:
        sleep()
        total += number
        print(f'작업중={name}, number={number}, total={total}')
    end = time.time()
    print(f'작업명={name}, 걸린시간={end - start}')
    return total

def main():
    start = time.time()
    result1 = sum('A', [1,2])
    result2 = sum('B', [1,2,3])
    end = time.time()
    print(f'총합={result1+result2}, 총시간={end-start}')

if __name__ == '__main__':
    main()

작업중=A, number=1, total=1
작업중=A, number=2, total=3
작업명=A, 걸린시간=2.015300989151001
작업중=B, number=1, total=1
작업중=B, number=2, total=3
작업중=B, number=3, total=6
작업명=B, 걸린시간=3.026836633682251
총합=9, 총시간=5.042137622833252


In [ ]:
# 풀이
# 다음은 앞의 코드에 asyncio 모듈을 적용한 문제 풀이이다.

# [파일명: asyncio_sample.py]

# import asyncio
# import time


# async def sleep():
#     await asyncio.sleep(1)


# async def sum(name, numbers):
#     start = time.time()
#     total = 0
#     for number in numbers:
#         await sleep()
#         total += number
#         print(f'작업중={name}, number={number}, total={total}')
#     end = time.time()
#     print(f'작업명={name}, 걸린시간={end-start}')
#     return total


# async def main():
#     start = time.time()

#     task1 = asyncio.create_task(sum("A", [1, 2]))
#     task2 = asyncio.create_task(sum("B", [1, 2, 3]))

#     await task1
#     await task2

#     result1 = task1.result()
#     result2 = task2.result()

#     end = time.time()
#     print(f'총합={result1+result2}, 총시간={end-start}')


# if __name__ == "__main__":
#     asyncio.run(main())

# 함수를 비동기로 호출하려면 이렇게 def 앞에 async라는 키워드를 넣으면 된다. 
# 그러면 이제 이 함수는 비동기 함수가 된다. 이때 async를 적용한 비동기 함수를 코루틴이라 부른다.

# 또한, 코루틴 안에서 다른 코루틴을 호출할 때는 await sleep()과 같이 await를 함수명 앞에 붙여 호출해야 한다. 
# 코루틴 수행 중 await 코루틴을 만나면 await로 호출한 코루틴이 종료될 때까지 기다리지 않고 제어권을 메인 스레드나 다른 코루틴으로 넘긴다. 
# 이러한 방식을 넌블록킹(non-blocking)이라 한다. 그리고 호출한 코루틴이 종료되면 이벤트에 의해 다시 그 이후 작업이 수행된다.

# 여기서 하나 눈여겨봐야 할 점은 sleep() 함수에서 time.sleep(1) 대신 asyncio.sleep(1)를 사용한 부분이다. 
# 코루틴이 아닌 time.sleep(1)을 사용한다면 await가 적용되지 않아 실행 시간을 줄일 수 없다.

# main() 함수에서 사용한 asyncio.create_task()는 수행할 코루틴 작업(태스크)을 생성한다. 
# 여기서는 작업을 생성할 뿐이지 실제로 코루틴이 수행되는 것은 아니다. 
# 실제 코루틴 실행은 await 태스크가 담당한다. 그리고 실행 태스크의 결괏값은 태스크.result()로 얻을 수 있다.

# asyncio.create_task()는 코루틴을 동시에 실행하는 데 꼭 필요하다. 
# 다음처럼 태스크가 아닌 await로 코루틴을 실행한다면 코루틴이 동시에 실행되지 않고 하나씩 차례로 실행되어 이득이 없을 것이다.

# result1 = await sum("A", [1, 2])
# result2 = await sum("B", [1, 2, 3])

# asyncio.run(main())은 런 루프를 생성하여 main() 코루틴을 실행한다. 
# 코루틴을 실행하려면 런 루프가 반드시 필요하다. 코루틴이 모두 비동기적으로 실행되기 때문에 그 시작과 종료를 감지할 수 있는 이벤트 루프가 반드시 필요하기 때문이다.

# 이 코드를 실행한 결과는 다음과 같다.

# c:\projects\pylib>python asyncio_sample.py
# 작업중=A, number=1, total=1
# 작업중=B, number=1, total=1
# 작업중=A, number=2, total=3
# 작업명=A, 걸린시간=2.000617742538452
# 작업중=B, number=2, total=3
# 작업중=B, number=3, total=6
# 작업명=B, 걸린시간=3.000927209854126
# 총합=9, 총시간=3.000927209854126

# A 작업과 B 작업을 교대로 호출한다. (제어권이 await에 의해 계속 바뀐다는 것을 알 수 있다.) 
# 그리고 시간도 5초 걸리던 것이 3초만 걸리게 되므로 A, B 작업이 완전히 비동기적으로 동작했다는 것을 알 수 있다.

# 참고
# asyncio - 비동기 I/O: https://docs.python.org/ko/3/library/asyncio.html
# 코루틴과 태스크: https://docs.python.org/ko/3/library/asyncio-task.html

## 072 서버와 통신하는 게임을 만들려면? ― socket

In [ ]:
# 072 서버와 통신하는 게임을 만들려면? ― socket
# socket은 TCP 서버/클라이언트 프로그램을 작성할 때 사용하는 모듈이다.

# 문제
# 서버에서 1~9 사이의 숫자를 무작위로 생성하고 클라이언트가 접속하여 그 숫자를 맞추는 게임을 
# socket 모듈을 사용하여 프로그래밍하고자 한다. 어떻게 해야 할까?

# 숫자 맞추기 게임의 규칙은 다음과 같다.

# 1. 서버에서 1~9 사이의 숫자(정답)를 무작위로 생성하고 클라이언트의 접속을 기다린다.
# 2. 클라이언트는 서버에 접속하여 1~9 사이의 값을 입력하여 게임을 시작한다.
# 3. 서버는 클라이언트가 입력한 숫자가 정답보다 높을 때는 "너무 높아요"라고, 낮을 때는 "너무 낮아요"라고 응답한다.
# 4. 클라이언트가 0을 입력하면 "종료"라고 응답하고 서버를 종료한다.
# 5. 클라이언트가 정답을 입력하면 "정답"이라고 응답하고 서버를 종료한다.

In [ ]:
# 풀이
# 소켓 서버
# 숫자 게임 서버는 다음과 같이 만든다.

# [파일명: socket_server.py]

# import socket
# import random

# HOST = ''
# PORT = 50007

# with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
#     s.bind((HOST, PORT))
#     s.listen()
#     print('서버가 시작되었습니다.')
#     conn, addr = s.accept()
#     with conn:
#         answer = random.randint(1, 9)
#         print(f'클라이언트가 접속했습니다:{addr}, 정답은 {answer} 입니다.')
#         while True:
#             data = conn.recv(1024).decode('utf-8')
#             print(f'데이터:{data}')

#             try:
#                 n = int(data)
#             except ValueError:
#                 conn.sendall(f'입력값이 올바르지 않습니다:{data}'.encode('utf-8'))
#                 continue

#             if n == 0:
#                 conn.sendall(f"종료".encode('utf-8'))
#                 break
#             if n > answer:
#                 conn.sendall("너무 높아요".encode('utf-8'))
#             elif n < answer:
#                 conn.sendall("너무 낮아요".encode('utf-8'))
#             else:
#                 conn.sendall("정답".encode('utf-8'))
#                 break

# 먼저 socket.socket(socket.AF_INET, socket.SOCK_STREAM)으로 소켓 객체 s를 생성한다. 
# 첫 번째 매개변수 socket.AF_INET은 IPv4 인터넷 프로토콜을, 두 번째 매개변수 socket.SOCK_STREAM은 소켓 유형이 문자열 등을 주고받는 스트림 방식임을 의미한다.

# s.bind((HOST, PORT))는 소켓 서버가 HOST라는 IP 주소의 PORT 번호에 해당하는 포트로 연결되도록 설정한다는 뜻이다. 
# HOST에는 빈 문자열을 지정했으므로 외부 접속을 허용한다. 만약 HOST에 빈 문자열 대신 'localhost'로 설정한다면 로컬 접속만 허용한다.

# s.listen() 은 서버 소켓이 클라이언트와의 연결을 시작할 수 있도록 바인딩된 포트를 연다. conn, addr = s.accept()는 클라이언트가 접속하면 연결을 수락하고 conn, addr을 반환한다. 이때 conn은 서버와 클라이언트가 연결된 소켓을 의미하고 addr은 클라이언트의 접속 IP를 의미한다.

# 클라이언트가 전송한 값을 수신하려면 conn.recv() 함수를 사용하고 클라이언트에 값을 송신하려면 conn.sendall() 함수를 사용한다. 이때 주고받는 메시지는 바이트 문자열이어야 하므로 보낼 때는 UTF-8로 인코딩하고 받을 때에도 UTF-8로 디코딩해야 한다.

# 참고: 부록 - 01 파이썬과 유니코드

# conn.recv(1024)에서 1024는 한 번에 수신받을 데이터의 최대 바이트 수를 의미하며 보통 1킬로바이트를 의미하는 1,024바이트를 사용한다. 만약 클라이언트로부터 1,024바이트 이상의 데이터를 수신해야 한다면 더 큰 숫자를 사용하거나 루프를 사용하여 conn.recv() 함수를 여러 번 수행해야 한다.

# 이후 클라이언트가 정답 또는 0을 입력할 때까지 클라이언트와 데이터를 주고받는다.

In [2]:
%%writefile socket_server.py
# socket_server.py

import socket
import random

HOST = ''
PORT = 50007

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    print('서버가 시작되었습니다.')
    conn, addr = s.accept()
    with conn:
        answer = random.randint(1, 9)
        print(f'클라이언트가 접속했습니다:{addr}, 정답은 {answer} 입니다.')
        while True:
            data = conn.recv(1024).decode('utf-8')
            print(f'데이터:{data}')

            try:
                n = int(data)
            except ValueError:
                conn.sendall(f'입력값이 올바르지 않습니다:{data}'.encode('utf-8'))
                continue

            if n == 0:
                conn.sendall(f"종료".encode('utf-8'))
                break
            if n > answer:
                conn.sendall("너무 높아요".encode('utf-8'))
            elif n < answer:
                conn.sendall("너무 낮아요".encode('utf-8'))
            else:
                conn.sendall("정답".encode('utf-8'))
                break

Writing socket_server.py


In [ ]:
# 소켓 클라이언트
# 숫자 게임 클라이언트는 다음과 같이 만든다.

# [파일명: socket_client.py]

# import socket

# HOST = 'localhost'
# PORT = 50007

# with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
#     s.connect((HOST, PORT))

#     while True:
#         n = input("1-9 사이의 숫자를 입력하세요(0은 게임포기):")
#         if not n.strip():
#             print("입력값이 잘못되었습니다.")
#             continue
#         s.sendall(n.encode('utf-8'))
#         data = s.recv(1024).decode('utf-8')
#         print(f'서버응답:{data}')
#         if data == "정답" or data == "종료":
#             break
# 소켓 클라이언트는 소켓 서버보다 간단하다. 가장 큰 차이점은 서버 소켓에서는 연결된 클라이언트를 의미하는 conn 객체를 통해 데이터를 주고받았다면 소켓 클라이언트에서는 서버와 연결을 의미하는 소켓 객체인 s 객체를 통해 데이터를 주고받는다는 점이다.

# 소켓 클라이언트는 서버가 아니므로 HOST = 'localhost'와 같이 접속할 HOST를 정확하게 입력해야 한다. 그리고 서버에 접속하려면 s.connect ((HOST, PORT))와 같이 connect() 함수를 통해 접속해야 한다.

# s.connect((HOST, PORT)) 함수에 전달하는 인수가 HOST, PORT가 아닌 (HOST, PORT)와 같은 튜플임에 주의하자.

# 그리고 서버 소켓과 데이터를 주고받고자 소켓 객체의 s.recv(), s.sendall() 함수를 사용한다. 사용 방법은 서버 소켓과 마찬가지이다.

# 동작확인
# 소켓 서버를 실행하고 나서 소켓 클라이언트를 실행하여 게임을 진행하고 동작을 확인해 보자.

# 서버소켓 실행 예

# c:\projects\pylib>python socket_server.py
# 서버가 시작되었습니다.
# 클라이언트가 접속했습니다:('127.0.0.1', 14761), 정답은 4 입니다.
# 데이터:1
# 데이터:9
# 데이터:5
# 데이터:3
# 데이터:4
# 클라이언트소켓 실행 예

# c:\projects\pylib>python socket_client.py
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):1
# 서버응답:너무 낮아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):9
# 서버응답:너무 높아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):5
# 서버응답:너무 높아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):3
# 서버응답:너무 낮아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):4
# 서버응답:정답
# 참고
# socket - 저수준 네트워킹 인터페이스: https://docs.python.org/ko/3/library/socket.html

In [8]:
%%writefile socket_client.py 
# socket_client.py
import socket

HOST = 'localhost'
PORT = 50007

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))

    while True:
        n = input("1-9 사이의 숫자를 입력하세요(0은 게임포기):")
        if not n.strip():
            print("입력값이 잘못되었습니다.")
            continue
        s.sendall(n.encode('utf-8'))
        data = s.recv(1024).decode('utf-8')
        print(f'서버응답:{data}')
        if data == "정답" or data == "종료":
            break

Overwriting socket_client.py
